# INTELIGENCIA ARTIFICIAL
## Tarea 2: implementacion de K-Means
### Hecho por: Juan Sebastián Clavijo Martínez
TEMA: Usar el dataset sobre datos de clientes de un centro comercial para implementar el metodo de clustering k-means<br />
Fecha: 01-09-2024<br />
Notas: el dataset fue obtenido de: https://github.com/tugrulhkarabulut/K-Means-Clustering/blob/master/example_datasets/Mall_Customers.csv<br />
**Pontificia Universidad Javeriana**

In [1]:
# Importado de bibliotecas pertinentes
# se usa el #type: ignore para que mi vscode genere warnings sobre la resolucion de dependencias de los pkts

import pandas as pd  # type: ignore # tratamiento de datos en un dataframe
import numpy as np  # type: ignore # numerical python
import seaborn as sns  # type: ignore # biblioteca versatil para estadistica y visualizacion
import matplotlib.pyplot as plt  # type: ignore # biblioteca para vizualizacion de datos
from ipywidgets import widgets, interactive, HBox # widgets interactivos de python
from IPython.display import display  # display interactivo

## Parte 1: preprocesamiento de datos

In [2]:
# Parte 1.1: carga de datos
# se trae el dataset, se puede ver en: https://github.com/tugrulhkarabulut/K-Means-Clustering/blob/master/example_datasets/Mall_Customers.csv
url = "https://raw.githubusercontent.com/tugrulhkarabulut/K-Means-Clustering/master/example_datasets/Mall_Customers.csv"
df_cust = pd.read_csv(url) # se llama así por la información que tiene, sobre varios indicadores de clientes de un centro comercial
# se presentan los primeros 5 renglones
df_cust.head(5)

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [3]:
# Parte 1.2: limpieza y preparación de los datos (cuenta de datos null, etc. y limpieza)
desaparecidos = len(df_cust) - len(df_cust.dropna())
Cantidad = len(df_cust)
print("Cantidad de datos observados con datos NaN", desaparecidos)
print("Cantidad de datos duplicados", df_cust.duplicated().sum())
##se eliminan los datos Null y Duplicados
df_cust.dropna(inplace=True)
df_cust.drop_duplicates(inplace=True)
# Se reinicia el indice por los elementos eliminados
df_cust.reset_index(drop=True, inplace=True)

Cantidad de datos observados con datos NaN 0
Cantidad de datos duplicados 0


## Parte 2: implementación de k-means

- Es importante recordar que k-means estima la media de la distribucion
- se habla de grupos (conjunto de elementos que son similares) no de clases
- se usa una metrica de distancia
- k es el numero de grupos que se tiene, es un hiperparametro de entrada
- PASOS
    1. definir k puntos como los centroides (media de los grupos) de los grupos a encontrar
    2. ya con los centroides colocados aleatoriamente, hallar distancia de cada punto a cada uno de los centroides
    3. asignar a cada elemento el grupo asociado con el centroide mas cercano (recordar distancia euclidiana)  [si hay solapamiento, la region de desicion que se toma es la mitad]
    4. recalcular el centroide como el valor prom. entre los elementos que pertenecen a cada grupo
    5. reasignar grupos en funcion de la distancia
    6. goto 3 hasta convergencia (no hay cambios/son minimos en el valor de las medias)
- criterios de parada:
  - numero de elementos en cada grupo, no cambia

In [4]:
# Parte 2.1 Implementación de k-means

#preparacion de columnas para dropdown
x_columns = df_cust.columns.drop(["CustomerID", "Gender", "Spending Score (1-100)"])

# Sliders para seleccionar el número de centroides (k) y el número de iteraciones (iter)
k_slider = widgets.IntSlider(value=3, min=1, max=10, step=1, description="Centroides:")
iter_slider = widgets.IntSlider(
    value=100,
    min=1,
    max=10000,
    step=1,
    description="Iteraciones:"
)

# Dropdown para seleccionar la columna del eje X
x_dropdown = widgets.Dropdown(
    options=x_columns, value="Annual Income (k$)", description="Eje X:"
)


# Función para crear centroides iniciales aleatorios
def creacionCentroides(k, data):
    n_samples, n_features = data.shape
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    return centroids


# Función para calcular la distancia euclidiana entre los puntos y los centroides
def calcularDistancia(data, centroids):
    distances = np.linalg.norm(data[:, np.newaxis] - centroids, axis=2)
    return distances


# Función para asignar los puntos al grupo más cercano basado en la distancia
def asignarGrupos(distances):
    return np.argmin(distances, axis=1)


# Función para recalcular los centroides como la media de los puntos asignados a cada grupo
def recalcularCentroides(data, grupos, k):
    new_centroids = np.array([data[grupos == i].mean(axis=0) for i in range(k)])
    return new_centroids


# Algoritmo de k-means
def kMeans(data, k, max_iter=100):
    centroids = creacionCentroides(k, data)
    for _ in range(max_iter):
        old_centroids = centroids
        distances = calcularDistancia(data, centroids)
        grupos = asignarGrupos(distances)
        centroids = recalcularCentroides(data, grupos, k)

        # Verifica si los centroides no cambian (convergencia)
        if np.all(old_centroids == centroids):
            break

    return centroids, grupos


# Función para graficar los resultados del clustering
def plot_graph(x_column, k, max_iter):
    data = df_cust[[x_column, "Spending Score (1-100)"]].values
    centroids, grupos = kMeans(data, k, max_iter)

    plt.figure(figsize=(8, 8))
    plt.scatter(
        data[:, 0], data[:, 1], c=grupos, alpha=0.5, edgecolor="none", cmap="viridis"
    )
    plt.scatter(
        centroids[:, 0], centroids[:, 1], s=100, c="red", marker="o"
    )  # Centroides marcados en rojo
    plt.xlabel(x_column)
    plt.ylabel("Spending Score (1-100)")
    plt.title(f"K-Means Clustering con k={k} e iteraciones={max_iter}")
    plt.show()


# Conectar el slider con la gráfica interactiva
interactive_plot = interactive(
    plot_graph, x_column=x_dropdown, k=k_slider, max_iter=iter_slider
)
display(interactive_plot)

interactive(children=(Dropdown(description='Eje X:', index=1, options=('Age', 'Annual Income (k$)'), value='An…